Notebook Linear Regression Case
Oefening Data Scientist 
Geert Vandezande

Doel:
- Supervised Learning toepassen
- EDA uitvoeren op een dataset
- Lineair Regression toepassen op de data: de target is beter doen dan r² = 80% nauwkeurigheid die in de meeste uitwerkingen zit...
- Ook door andere vormen van regressie toe te passen, het doel is om r² zo goed mogelijk te krijgen



Extra:
- er wordt logging voorzien voor en na de belangrijke stappen (zie LinReg_logging.log). Hiermee kunnen de stappen en de resultaten opgevolgd worden. Dit wordt in de LinReg_logging weg geschreven
- we hebben een aantal herbruikbare code-blokken in een functie gestoken
- een aparte class gemaakt voor BinaryValueEncoders om eens te proberen (kan uiteraard met de OneHotEncoder)
- we hebben een functie geschreven om snel een reeks van modellen te kunnen evalueren, zowel zonder pipelining als met pipelining

Dataset: 
- More info: see kaggle https://www.kaggle.com/datasets/mirichoi0218/insurance/data

Metadata :
- age: age of primary beneficiary
- sex: insurance contractor gender, female, male
- bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height, objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9
- children: Number of children covered by health insurance / Number of dependents
- smoker: Smoking
- region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.
- charges: Individual medical costs billed by health insurance


Volgorde van activiteiten in deze notebook: (cfr Datacamp "preparing data for modelling)
- data inlezen
- data bekijken, visueel en numerisch
- data summarizen via summarytools 
- missing en duplicated data oplossen 
- incorrect types controleren
- numerische waarde standardizeren
- categorische varaiabelen processen
- feature engineering checken
- linear, ridge, lasso, gradient boost, random forest,...  modellen uitvoeren
- alle modellen worden toegepast op 2 situaties voor stratefy
    - Train_test_split zonder stratify
    - Train_test_split met stratefy op basis van de categorisatie van de charges (target v) (omdat er skewness is op charges )

- zowel met hyper parameter tuning en zonder

- extra test: met verwijderen van outliers in BMI ==> geen beter resultaat
- enkel rokers: 





In [ ]:
# import van de diverse modules
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from scipy.stats.mstats import winsorize



# Machine learning algorithm
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import GridSearchCV

# Evaluation
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error



# system utils
import warnings
from pathlib import Path
import datetime
from colorama import Fore, Back, Style
import sys
import os
import chardet
from tabulate import tabulate

Extra code snippits die doorheen de notebook gebruikt worden:

save_fig: na generatie van een image kan de image naar file geschreven worden in de images/.. directory. Geef steeds een zinvolle naam

read_JSON: om eenvoudig een JSON in te lezen

log_info:
- logging functie om doorheen de notebooks de status naar file te kunnen schrijven. 
- de logstatements worden tijdens de uitvoering van de code bewaard in een list. Die kan tussentijds naar het scherm geprint worden of naar een file
- log_info_write_to_file: schrijf de loginformatie naar file 
- log_info_print_on_screen: print alle loginfo naar het scherm

In [ ]:
# enkele extra code snippets gebruikt doorheen de oefening

# to plot or not to plot - zet op True om de plots te zien, zet op False om de plots niet te zien bij een Run ALL
plot_graphs = True

# schrijf een visual naar file

IMAGES_PATH = Path() / "images" 
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Lezen van de JSON-file

def read_JSON(file_path_read):
    with open(file_path_read, 'r') as file:
        files_from_json = json.load(file)
    return files_from_json


# functies om te loggen naar file
log_info_lijst = []
log_filenaam = "LinReg_continue.log"

if os.path.exists(log_filenaam):
    os.remove(log_filenaam)

def log(log_code="INFO", boodschap="euh geen boodschap????"):
    global log_info_lijst
    now = datetime.datetime.now()
    formatted_date = now.strftime("%d/%m/%Y %H:%M:%S")
    log_message = f"{Style.RESET_ALL}{formatted_date} : {log_code} : {boodschap}"
    log_info_lijst.append(log_message)
    with open(log_filenaam, 'a') as file:  # Open the file in append mode
        file.write(str(boodschap) + '\n')  # Voeg een nieuwe regel toe na elke string
    print(log_message)
    return

def log_info(boodschap):
    log("Info",boodschap)
    def log_info_write_to_file(filename):
        with open(filename, 'a') as file:  # Open the file in append mode
            for string in log_info_lijst:
                file.write(string + '\n')  # Voeg een nieuwe regel toe na elke string
        return

def log_info_write_to_file(filename):
    with open(filename, 'w') as file:
        for string in log_info_lijst:
            file.write(string + '\n')  # Voeg een nieuwe regel toe na elke string
    return

def log_info_print_on_screen():
    for boodschap in log_info_lijst:
        print(boodschap)    
    return

Code snippets om visuals op het scherm te plaatsen
Deze worden doorheen de notebook gebruikt

In [ ]:
# functie: maakt een boxplot van kolommen in een pandaframe
# df_col is een list van de kolomnamen die geplot worden
def plot_boxplot(df, df_col, filenaam):
    if plot_graphs:
        # boxplot van de numerische waarden
        sns.set_theme(style="whitegrid", palette="bright")
        plt.figure(figsize=(15, 10)) 
        for i, col in enumerate(df_col):
            plt.subplot(len(df_col), 2, 2 * i + 1)
            sns.boxplot(x=df[col], orient='h', linewidth=1.5)
            plt.title(f"Boxplot of {col}", fontsize=12, fontweight="bold")
            plt.xlabel(col, fontsize=10)

            plt.subplot(len(df_col), 2, 2 * i + 2)
            sns.histplot(df[col], kde=True,  linewidth=1)
            plt.title(f"Distribution Plot of {col}", fontsize=12, fontweight="bold")
            plt.xlabel(col, fontsize=10)
            plt.ylabel("Density", fontsize=10)

        plt.tight_layout()
        save_fig(filenaam)
        plt.show()

Hulpfuncties

bereken_percentage_outliers: via Isolation forest wordt het percentage van de outliers berekend.
- df : dataframe
- columns_to_use: list van koloms die gebruikt worden om de outliers te berekenen
- functie geeft een percentage terug van het aaantal outliers op het totaal aantal observaties


cap_values: vervang outliers door hun lower of upperpercentieel waarde: de whiskers worden berekend door van de lower_percentieel waarde een waarde af te trekken gelijk aan 1,5 * IQR (interquartile range), voor upper_percentieel waarde wordt de 1,5 * IQR bijgeteld
- df: dataframe
- columns_to_use
- lower_percentieel (default = 25)
- upper_percentieel (default = 75)


BinaryValueEncoder: hulp klasse om categorische te classificeren naar 0 en 1
- string_zero: indien categorische waarde = string_zero, krijgt de nieuwe waarde 0
- string_one: indien categorische waarde = string_zero, krijgt de nieuwe waarde 1
Try-out om zelf eens een class te schrijven voor transformation. Kan uiteraard ook via OneHotEncoding gedaan worden


In [ ]:
# functie om het percentage outliers te berkenen voor een set van kolommen in een dataframe
def bereken_percentage_aantal_outliers(df , columns_to_use):
    # Initialiseren van het Isolation Forest model
    iso_forest = IsolationForest(n_estimators=100, contamination='auto', random_state=42)

    # Fit het model
    iso_forest.fit(df[columns_to_use])
    # Voorspellingen
    # Het geeft -1 voor outliers en 1 voor inliers
    labels = iso_forest.predict(df[columns_to_use])
    # Toevoegen van de labels aan het DataFrame om outliers te identificeren
    df_intern = df.copy()
    df_intern['outlier'] = labels
    outliers = df_intern[df_intern['outlier'] == -1]
    aantal_outliers = df_intern['outlier'].value_counts()
    print(aantal_outliers)
    percentage_aantal_outliers = (len(outliers) / len(df_intern)) * 100

    return percentage_aantal_outliers

# functie om outliers in een kolom te cappen op een percentiel waarde
def cap_values(df_input, column, lower_percentile=25, upper_percentile=75):
    # voeg code toe om beter de outliers te verwijderen
    log("Info", f"Capping values voor kolom {column} naar lower percentiel {lower_percentile} - upper percentiel {upper_percentile}")
    q1, q3 = np.percentile(df_input[column], [lower_percentile, upper_percentile])  # Calculate the 25th (Q1) and 75th (Q3) percentiles
    iqr = q3 - q1  # Calculate the interquartile range (IQR)
    lower_bound = q1 - 1.5 * iqr  # Calculate lower whisker (Q1 - 1.5 * IQR)
    upper_bound = q3 + 1.5 * iqr  # Calculate upper whisker (Q3 + 1.5 * IQR)

    # lower_bound = df[column].quantile(lower_percentile)
    # upper_bound = df[column].quantile(upper_percentile)
    
    # Waarden cappen met behulp van de numpy.where functie
    df_output = df_input.copy()
    df_output[column] = np.where(df_input[column] < lower_bound, lower_bound, df_input[column])
    df_output[column] = np.where(df_input[column] > upper_bound, upper_bound, df_input[column])    
    return df_output


# hulp klasse om categorische waarden met twee mogelijke waarde naar 0 en 1 om te zetten
# was een try-out om zelf eens een encoder te schrijven
# kan uiteraard eenvoudiger door OneHotEncoding toe te passen, we hebben dan ook OneHotEncoding toegepast 
class BinaryValueEncoder(TransformerMixin, BaseEstimator):
    def __init__(self, string_zero="nul", string_one="een"):
        # Je kunt hier extra initialisatie toevoegen indien nodig
        self.string_zero = string_zero
        self.string_one = string_one
     

    def fit(self, X, y=None):
        # Er is geen fitting nodig voor deze eenvoudige codering
        return self

    def transform(self, X):
        # X wordt aangenomen een pandas DataFrame te zijn
        log("Info", f"BinaryValueEncoder transform opgeroepen voor One_value {self.string_one} en Zero_value {self.string_zero}")
        X = X.copy()  # Kopieer de DataFrame om wijzigingen te voorkomen in het origineel
        X = X.applymap(lambda x: 1 if x == self.string_zero else 0)
        return X
    
    def get_feature_names_out(self, input_features=None):
        # Dit is een eenvoudige passthrough-voorbeeld, waarbij feature namen niet wijzigen.
        if input_features is not None:
            return input_features
        else:
            return np.array(['x{}'.format(i) for i in range(X.shape[1])], dtype=object)


# hulpklasse om de waarden van een kolom te normaliseren en in een pipeline te gebruiken
class WinsorizeTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, limits=[0.05, 0.05]):
        self.limits = limits

    def fit(self, X, y=None):
        return self  # Niets te fitten, keert zichzelf terug

    def transform(self, X, y=None):
        return winsorize(X, limits=self.limits)
    
    def get_feature_names_out(self, input_features=None):
        # Genereer en retourneer de lijst met uitvoerfeaturenamen
        if input_features is None:
            input_features = [f'feature_{i}' for i in range(X.shape[1])]
        output_features = [f"{name}" for name in input_features]
        return output_features


Twee extra functies om een pipeline uit te voeren:
- één zonder hyperparameter tuning
- één met hyperparameter tuning

In [ ]:
# Functie om op basis van een X_train, X_test, y_train, y_test en de preprocessessin)
# return is een dataframe met de resultaten van de verschillende modellen
# zonder hyperparameter tuning
def excute_regression_models_with_pipeline(X_train, X_test, y_train,y_test,preprocessor):

    best_XGB_params = {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}
    
    models = {
        'Linear Regression': LinearRegression(),
        'Lasso': Lasso(alpha=0.1,random_state=42),
        'Ridge': Ridge(alpha=0.01,random_state=42),
        'Random Forest': RandomForestRegressor(random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(random_state=42), 
        'Decision Tree': DecisionTreeRegressor(random_state=42), 
        'XGBRegressor': XGBRegressor(random_state=42, **best_XGB_params)
    }

    results = []
    for name, model in models.items():        
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('poly', PolynomialFeatures(degree=2, include_bias=False)),
            ('regressor', model)
        ]) 
        
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)

        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        test_r2 = r2_score(y_test, y_pred)
        test_mape = 100 * mean_absolute_percentage_error(y_test, y_pred)

        results.append([name, test_rmse, test_r2, test_mape])
    df = pd.DataFrame(results, columns=['model', 'test_rmse', 'test_r2', 'test_mape'])

    return df


# Functie om op basis van een X_train, X_test, y_train, y_test en de preprocessor
# return is een dataframe met de resultaten van de verschillende modellen
# met hyperparameter tuning
def execute_regression_models_with_pipeline_hyperparameter_tuning(X_train, X_test, y_train, y_test, preprocessor):
    # Hyperparameters sets voor verschillende modellen

    models = {
        'Linear Regression': LinearRegression(),
        'Lasso': Lasso(alpha=0.1,max_iter=10000, random_state=42),
        'Ridge': Ridge(alpha=0.01,random_state=42),
        'Random Forest': RandomForestRegressor(random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(random_state=42), 
        'Decision Tree': DecisionTreeRegressor(random_state=42), 
        'XGBRegressor': XGBRegressor(random_state=42)
    }

    param_grid = {
        'Lasso': {'regressor__alpha': [0.01, 0.1, 1.0]},
        'Ridge': {'regressor__alpha': [0.01, 0.1, 1.0]},
        'Random Forest': {'regressor__n_estimators': [50, 100, 200], 'regressor__max_depth': [None, 10, 20]},
        'Gradient Boosting': {'regressor__n_estimators': [100, 200], 'regressor__learning_rate': [0.1, 0.05, 0.01]},
        'Decision Tree': {'regressor__max_depth': [None, 10, 20]},
        'XGBRegressor': {'regressor__n_estimators': [50, 100], 'regressor__max_depth': [3, 5], 'regressor__learning_rate': [0.1, 0.05]}
    }

    results = []
    for name, model in models.items():
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('poly', PolynomialFeatures(degree=2, include_bias=False)),
            ('regressor', model)
        ])

        # Als het model tuning vereist, gebruik GridSearchCV
        if name in param_grid:
            grid_search = GridSearchCV(pipeline, param_grid[name], cv=5, scoring='neg_mean_squared_error')
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_
            y_pred = best_model.predict(X_test)
        else:
            pipeline.fit(X_train, y_train)
            y_pred = pipeline.predict(X_test)

        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        test_r2 = r2_score(y_test, y_pred)
        test_mape = 100 * mean_absolute_percentage_error(y_test, y_pred)

        results.append([name, test_rmse, test_r2, test_mape])

    df = pd.DataFrame(results, columns=['model', 'test_rmse', 'test_r2', 'test_mape'])
    return df



Hier beginnen we er echt met het inlezen van de data en de eerste checks op de data

In [ ]:
# data bestand inlezen

insurance_data_filename = 'data/insurance.csv'
df = pd.read_csv(insurance_data_filename)
log_info(f"File ingelezen: {insurance_data_filename}")

# check op duplicates, indien zo verwijder direct
df.drop_duplicates(inplace=True)
df_original = df.copy()
duplicate_waarden = df.duplicated().sum()
log_info(f"Check op duplicates na drop \n{duplicate_waarden}")

# behoudt een copie van de orginele data
df_original = df.copy()

In [ ]:
# enkele eenvoudige controles
log_info(f"df.info : \n{df.info()}")
log_info(f"df.describe : \n{df.describe()}")

# geen nulwaarden 

from summarytools import dfSummary
dfSummary(df)

In [ ]:
# drie categorische features: smoking, region en sex
# vier numerische features waaronder de target variabele "charges"
# maak de datasets aan 

df_cat_col = ['smoker','region','sex']
df_num_col = ['age', 'bmi','children']
df_label_col = ['charges']

# zijn er nominaal categorische variabelen en één ordinal categorische waarden?
df_cat_nom_col = ['smoker','region']
df_cat_ord_col = list(set(df_cat_col) - set(df_cat_nom_col))

df_num = df[df_num_col]
df_cat = df[df_cat_col]
df_label = df[df_label_col]
df_cat_nom = df[df_cat_nom_col]
df_cat_ord = df[df_cat_ord_col]

In [ ]:
# korte analyze van de categorische variabelen
print(df_cat.value_counts())

# check op nullen
nul_waarden = df.isnull().sum()
log_info(f"Check op nulwaarden \n{nul_waarden}")
# geen nullen

In [ ]:
plot_boxplot(df, df_num_col,"Boxplot en histogram van de numerische waarden")

In [ ]:
plot_boxplot(df, df_label_col,"Boxplot en histogram van de target value")

# we zien hier dat er enkele outliers zijn in de kolom charges
# we gaan hier rekening mee houden nadien en testen of het aanpakken van de outliers een beter model oplevert


In [ ]:
if plot_graphs:
    sns.set_theme(style="whitegrid", palette="bright")
    plt.figure(figsize=(15, len(df_cat_col) * 2))  

    for i, col in enumerate(df_cat_col):
        plt.subplot(1, len(df_cat_col), i + 1)
        sns.countplot(x=col, data=df, palette='bright', hue=col)
        plt.title(f"Count Plot of {col}", fontsize=14, fontweight="bold")
        plt.xlabel(col, fontsize=12)
        plt.ylabel("Count", fontsize=12)
        plt.xticks(rotation=45)

    plt.tight_layout()
    save_fig("Categorische features countplot")
    plt.show()
    log_info("Check van categorische features")

In [ ]:
if plot_graphs:
    sns.set_theme(style="whitegrid", palette="bright")
    plt.figure(figsize=(15, len(df_cat_col) * 2))  

    for i, col in enumerate(df_cat_col):
        plt.subplot(1, len(df_cat_col), i + 1)
        sns.countplot(x=col, data=df, palette='bright', hue='smoker')
        plt.title(f"Count Plot of {col} (Hue: Smoker)", fontsize=14, fontweight="bold")
        plt.xlabel(col, fontsize=12)
        plt.ylabel("Count", fontsize=12)
        plt.xticks(rotation=45)

    plt.tight_layout()
    save_fig("Categorische variabelen tov smoker")
    plt.show()

In [ ]:
if plot_graphs:
    sns.pairplot(df, hue='smoker',  kind='reg')
    plt.tight_layout()
    save_fig("Numerische features onderlinge scatter")
    plt.show()

In [ ]:
# print de correlation nog af tussen de numerische waarden
if plot_graphs:
    Dataset = df.copy()
    Dataset['sex'] = Dataset['sex'].replace(['male','female'],[0,1])
    Dataset['smoker'] = Dataset['smoker'].replace(['yes','no'],[1,0])
    Dataset['region'] = Dataset['region'].replace(['southwest','southeast','northwest','northeast'],[0,1,2,3])

    sns.heatmap(Dataset.corr(),annot=True,cmap='Blues')
    plt.title('correlation of Insurance Data')
    plt.show()


# enkel tussen smoker en charges is er een duidelijke correlatie

In [ ]:
# Feature preparation - implementatie van een preprocessor pipeline
# standard scaler op de numerische waarden, min-max scaler
# preprocessor met basic scalen
# one hot encoding voor de categorische waarden

num_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("standardize", StandardScaler()),
])

male_female_transformer = Pipeline(steps=[   
    ('male_female_encoder', OneHotEncoder(drop='first'))
])

smoking_transformer = Pipeline(steps=[  
    ('smoking_encoder', OneHotEncoder(drop='first'))
])

regio_transformer = Pipeline(steps=[
    ('regio', OneHotEncoder(drop='first', handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", num_pipeline, df_num_col),
    ("male_female", male_female_transformer, ['sex']), 
    ("smoker", smoking_transformer, ['smoker']), 
    ("regio", regio_transformer, ['region'])],
     remainder='passthrough')

df_features = df_original.drop(['charges'], axis= 1)
np_prepared =  preprocessor.fit_transform(df_features)

df_prepared = pd.DataFrame(
    np_prepared,
    #columns=preprocessing.get_feature_names_out(),
    columns = [name.split('__')[-1] for name in preprocessor.get_feature_names_out()],    
    index=df_original.index)

df_prepared.head()

plot_boxplot(df_prepared,df_num_col,"Boxplot van numerische waarden na standard scaling")




In [ ]:
# We maken ook een preprocessor om de BMI-waarden te cappen
# met behulp van een eigen transformer winsorize 

num_pipeline_zonder_BMI = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("standardize", StandardScaler()),
])


num_pipeline_BMI = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ('winsorizer', WinsorizeTransformer()),
    ("standardize", StandardScaler()),
])

df_num_col_zonder_bmi = ['age', 'children']

male_female_transformer = Pipeline(steps=[    
    ('male_female_encoder', OneHotEncoder(drop='first'))
])

smoking_transformer = Pipeline(steps=[    
    ('smoking_encoder', OneHotEncoder(drop='first'))
])

regio_transformer = Pipeline(steps=[
    ('regio', OneHotEncoder(drop='first', handle_unknown="ignore"))
])

preprocessor_bmi = ColumnTransformer([
    ("num", num_pipeline_zonder_BMI, df_num_col_zonder_bmi),
    ("BMI", num_pipeline_BMI, ['bmi']),
    ("male_female", male_female_transformer, ['sex']), 
    ("smoker", smoking_transformer, ['smoker']), 
    ("regio", regio_transformer, ['region'])],
     remainder='passthrough')

df_features = df_original.drop(['charges'], axis= 1)
np_prepared =  preprocessor_bmi.fit_transform(df_features)

print(df_prepared.columns)

df_prepared = pd.DataFrame(
    np_prepared,
    #columns=preprocessing.get_feature_names_out(),
    columns = [name.split('__')[-1] for name in preprocessor_bmi.get_feature_names_out()],    
    index=df_original.index)

df_prepared.head()

plot_boxplot(df_prepared, df_num_col,"Boxplot van numerische waarden na standard scaling en robuust scaling van BMI")   

We gaan nu de train_test_split testen.
De "charges" zijn niet standard verdeeld. 
We gaan een stratify uitvoeren op de charges maar bins-gewijs.

Eerst checken we de gewone train_test_split zonder stratefy.
Dan checken we de train_test_split met stratefy

In [ ]:
# checken van de verdeling van de y_train en y_test

X = df_original.drop(['charges'], axis=1)
y = df_original['charges']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Plot histogrammen voor y_train en y_test
plt.figure(figsize=(12, 6))
plt.hist(y_train, bins=30, alpha=0.5, label='y_train', color='blue')
plt.hist(y_test, bins=30, alpha=0.5, label='y_test', color='red')
plt.title('Histogram van y_train en y_test')
plt.xlabel('Waarden')
plt.ylabel('Frequentie')
plt.legend()
plt.show()
save_fig("Histogram van y_train en y_test zonder stratefy")


In [ ]:
# checken van de verdeling van de y_train en y_test na stratify 

X = df_original.drop(['charges'], axis=1)
y = df_original['charges']
y_bins = pd.cut(y, bins=[0, 20000,30000,50000, np.inf], labels=[1, 2, 3, 4])
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y_bins, test_size=0.2, random_state=42)

# Plot histogrammen voor y_train en y_test
plt.figure(figsize=(12, 6))
plt.hist(y_train, bins=30, alpha=0.5, label='y_train', color='blue')
plt.hist(y_test, bins=30, alpha=0.5, label='y_test', color='red')
plt.title('Histogram van y_train en y_test')
plt.xlabel('Waarden')
plt.ylabel('Frequentie')
plt.legend()
plt.show()
save_fig("Histogram van y_train en y_test met stratefy")

We voeren nu de regression uit met behulp van de pipeline functie én voor 6 modellen.
De data is niet gestratified

In [ ]:
# eenvoudige regressie  zonder stratefy

X = df_original.drop(['charges'], axis=1)
y = df_original['charges']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
resultaat = excute_regression_models_with_pipeline(X_train, X_test, y_train, y_test, preprocessor)
resultaat_string = tabulate(resultaat, headers='keys', tablefmt='psql')

log_info("\n\n")
log_info("Regression models with polynomial features, no stratefy")
log_info("*************************************************************")
log_info(resultaat_string)
log_info("\n")


We voeren nu de regression uit met behulp van de pipeline functie én voor 6 modellen.
De data is nu wel gestratified

In [ ]:
X = df_original.drop(['charges'], axis=1)
y = df_original['charges']

y_bins = pd.cut(y, bins=[0, 20000,30000,50000, np.inf], labels=[1, 2, 3, 4])
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y_bins, test_size=0.2, random_state=42)
resultaat = excute_regression_models_with_pipeline(X_train, X_test, y_train, y_test, preprocessor)
resultaat_string = tabulate(resultaat, headers='keys', tablefmt='psql')

log_info("\n\n")
log_info("Regression models with polynomial features, met stratify\n")
log_info("Stratify on charges_cat: extra feature engineering: charges omzetten naar categories : bins=[0, 20000, 30000,50000, np.inf")
log_info("Regression models polynomial features en stratify on charges_cat\n\n")
log_info(resultaat_string)

En nu doen we hetzelfde met hyper parameter tuning.
Eens zonder en eens met stratify

In [ ]:
#zonder stratify
X = df_original.drop(['charges'], axis=1)
y = df_original['charges']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
resultaat_eenvoudig_met_pipeline = execute_regression_models_with_pipeline_hyperparameter_tuning(X_train, X_test, y_train, y_test, preprocessor)
resultaat_string = tabulate(resultaat_eenvoudig_met_pipeline, headers='keys', tablefmt='psql')

log_info("\n\n")
log_info("Regression models with polynomial features en hyperparameter tuning, met stratify\n")
log_info("Regression models polynomial features en stratify on charges_cat\n\n")
log_info(resultaat_string)

In [ ]:
X = df_original.drop(['charges'], axis=1)
y = df_original['charges']

y_bins = pd.cut(y, bins=[0, 20000,30000,50000, np.inf], labels=[1, 2, 3, 4])
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y_bins, test_size=0.2, random_state=42)
resultaat_eenvoudig_met_pipeline = execute_regression_models_with_pipeline_hyperparameter_tuning(X_train, X_test, y_train, y_test, preprocessor)
resultaat_string = tabulate(resultaat_eenvoudig_met_pipeline, headers='keys', tablefmt='psql')

log_info("\n\n")
log_info("Regression models with polynomial features en hyperparameter tuning, met stratify\n")
log_info("Stratify on charges_cat: extra feature engineering: charges omzetten naar categories : bins=[0, 20000, 30000,50000, np.inf")
log_info("Regression models polynomial features en stratify on charges_cat en met hyperparameter tuning\n\n")
log_info(resultaat_string)


Nu gaan we eens testen of er verbetering is als we de outliers in BMI oplossen

In [ ]:
X = df_original.drop(['charges'], axis=1)
y = df_original['charges']

y_bins = pd.cut(y, bins=[0, 20000,30000,50000, np.inf], labels=[1, 2, 3, 4])
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y_bins, test_size=0.2, random_state=42)

resultaat_eenvoudig_met_pipeline = execute_regression_models_with_pipeline_hyperparameter_tuning(X_train, X_test, y_train, y_test, preprocessor_bmi)
resultaat_string = tabulate(resultaat_eenvoudig_met_pipeline, headers='keys', tablefmt='psql')

log_info("\n\n")
log_info("Zonder outliers in BMI - Regression models with polynomial features, met stratefy\n")
log_info("Stratify on charges_cat: extra feature engineering: charges omzetten naar categories : bins=[0, 20000, 30000, np.inf")
log_info("Regression models polynomial features en stratify on charges_cat\n\n")
log_info(resultaat_string)


En nu nog een test om enkel niet-rokers en enkel rokers te modelleren

In [ ]:
# test om alle rokers te verwijderen en terug een check te doen

df_no_smokers = df_original[df_original['smoker'] == 'no']

X = df_no_smokers.drop(['charges'], axis=1)
y = df_no_smokers['charges']

y_bins = pd.cut(y, bins=[0, 20000,30000,50000, np.inf], labels=[1, 2, 3, 4])
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y_bins, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
resultaat = execute_regression_models_with_pipeline_hyperparameter_tuning(X_train, X_test, y_train, y_test, preprocessor)
resultaat_string = tabulate(resultaat, headers='keys', tablefmt='psql')

log_info("\n\n")
log_info("nExtra test: alle rokers verwijderd\n")
log_info("Regression models with polynomial features, met stratefy")
log_info("Stratify on charges_cat: extra feature engineering: charges omzetten naar categories : bins=[0, 20000, 30000, 50000, np.inf")
log_info("Regression models polynomial features en stratify on charges_cat\n\n")
log_info(resultaat_string)

In [ ]:
# test om alle rokers te verwijderen en terug een check te doen

df_no_smokers = df_original[df_original['smoker'] == 'yes']

X = df_no_smokers.drop(['charges'], axis=1)
y = df_no_smokers['charges']

y_bins = pd.cut(y, bins=[0, 20000,30000,50000, np.inf], labels=[1, 2, 3, 4])
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y_bins, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
resultaat = execute_regression_models_with_pipeline_hyperparameter_tuning(X_train, X_test, y_train, y_test, preprocessor)
resultaat_string = tabulate(resultaat, headers='keys', tablefmt='psql')

log_info("\n\n")
log_info("Extra test: alle rokers verwijderd\n")
log_info("Regression models with polynomial features, met stratefy")
log_info("Stratify on charges_cat: extra feature engineering: charges omzetten naar categories : bins=[0, 20000, 30000, 50000, np.inf")
log_info("Regression models polynomial features en stratify on charges_cat\n\n")
log_info(resultaat_string)

In [ ]:
# test om enkel te testen op BMI en charges

preprocessor_simple = ColumnTransformer([],
     remainder='passthrough')

df_no_smokers = df_original[df_original['smoker'] == 'no']

X = df_original['bmi'].values.reshape(-1, 1)
y = df_original['charges']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
resultaat = execute_regression_models_with_pipeline_hyperparameter_tuning(X_train, X_test, y_train, y_test, preprocessor_simple)
resultaat_string = tabulate(resultaat, headers='keys', tablefmt='psql')

log_info("Extra test: alle rokers verwijderd\n")
log_info("Regression models with polynomial features, met stratefy")
log_info("Stratify on charges_cat: extra feature engineering: charges omzetten naar categories : bins=[0, 20000, 30000, np.inf")
log_info("Regression models polynomial features en stratify on charges_cat\n\n")
log_info(resultaat_string)
